<a href="https://colab.research.google.com/github/manabtikadar/my_project/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [5]:
print(f"first few elements\n:{X_train[:5]}")
print(f"X_train data type:{type(X_train)}")

first few elements
:[[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
X_train data type:<class 'numpy.ndarray'>


In [6]:
print(f"first few elements:{y_train[:5]}")
print(f"y_train data type:{type(y_train)}")

first few elements:[1 1 0 0 1]
y_train data type:<class 'numpy.ndarray'>


In [7]:
def calculate_entropy(y):
    entropy = 0
    if len(y) != 0:
      p1 = len(y[y==1])/len(y)

      if (p1!=0)&(p1!=1):
         entropy = -p1*np.log2(p1)-(1-p1)*np.log2(1-p1)
      else:
         entropy = 0

    return entropy

In [8]:
print("Entropy at root node: ", calculate_entropy(y_train))

Entropy at root node:  1.0


In [24]:
def split_dataset(X, node_indices, feature):
  m,n = X.shape
  left_indices = []
  right_indices = []

  for i in range(m):
    if X[i,feature] == 1:
      left_indices.append(i)
    else:
      right_indices.append(i)

  return left_indices,right_indices

In [10]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

left_indices, right_indices = split_dataset(X_train, root_indices, feature=2)

print("left_indices:",left_indices)
print("right_indices:",right_indices)

left_indices: [0, 1, 4, 5, 7]
right_indices: [2, 3, 6, 8, 9]


In [11]:
def information_gain(X,y,node_indices,feature):
  left_indices, right_indices = split_dataset(X,node_indices,feature)
  left_entropy = calculate_entropy(y[left_indices])
  right_entropy = calculate_entropy(y[right_indices])
  node_entropy = calculate_entropy(y[node_indices])

  information_gain = 0.0

  w_left = len(left_indices)/len(node_indices)
  w_right = len(right_indices)/len(node_indices)

  information_gain = node_entropy -(w_left*left_entropy + w_right*right_entropy)

  return information_gain

In [12]:
info_gain0 = information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


In [20]:
def get_best_split(X,y,node_indices):
  num_features = X.shape[1]
  best_feature = -1
  max_information_gain = 0

  for feature in range(num_features):
    information_gain_ = information_gain(X,y,node_indices,feature)
    if information_gain_ > max_information_gain:
      max_information_gain = information_gain_
      best_feature = feature

  return best_feature

In [21]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [22]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.

    """

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices)
    tree.append((current_depth, branch_name, best_feature, node_indices))

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [27]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: -1
  -- Left leaf node with indices [0, 1, 4, 5, 7]
  -- Right leaf node with indices [2, 3, 6, 8, 9]
- Depth 1, Right: Split on feature: -1
  -- Left leaf node with indices [0, 1, 4, 5, 7]
  -- Right leaf node with indices [2, 3, 6, 8, 9]
